## CONTENT BASED FILTERING RECOMMENDATION SYSTEM 

### EUCLIDEAN DISTANCE GENERATOR 

#### LOAD THE LIBRARIES FOR MODEL

In [1]:
from __future__ import print_function
import pandas as pd
import re
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_columns = 80

2023-06-12 12:14:52.326247: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 12:14:52.328929: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-12 12:14:52.384048: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-12 12:14:52.385325: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 12:14:53.258655: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Instructions for updating:
non-resource variables are not supported in the long term


#### LOADING THE DATA AND DATA PRELIMINARY CLEANING 

In [2]:
Moviesdf = pd.read_csv ("imdb_processed.csv")
Moviesdf.head()

,Name,Year,kind,genre,imdb_rating,vote,country,language,cast,director,composer,writer,runtime
0,Dinosaur Planet,2003,tv mini series,"['Documentary', 'Animation', 'Family']",7.7,474.0,['United States'],['English'],"['Christian Slater', 'Scott Sampson']",NaN,NaN,"['Mike Carrol', 'Mike Carroll', 'Georgann Kane']",50.0
1,Get Up and Dance!,1994,video movie,['Family'],8.1,18.0,['United States'],['English'],"['Paula Abdul', 'Aurorah Allain', 'Bill Bohl',...",['Steve Purcell'],NaN,NaN,83.0
2,8 Man,1992,movie,"['Action', 'Sci-Fi']",5.5,93.0,['Japan'],['Japanese'],"['Kai Shishido', 'Etsushi Takahashi', 'Sachiko...",['Yasuhiro Horiuchi'],['Carole King'],"['Kazumasa Hirai', 'Jirô Kuwata', 'Junko Suzuki']",NaN
3,What the #$*! Do We (K)now!?,2004,movie,"['Documentary', 'Comedy', 'Drama', 'Fantasy', ...",5.3,13432.0,['United States'],"['English', 'German', 'Spanish']","['Marlee Matlin', 'Elaine Hendrix', 'John Ross...","['William Arntz', 'Betsy Chasse', 'Mark Vicente']",['Christopher Franke'],"['William Arntz', 'Betsy Chasse', 'Matthew Hof...",60.0
4,Class of Nuke 'Em High Part II: Subhumanoid Me...,1991,movie,"['Comedy', 'Horror', 'Sci-Fi']",4.5,2177.0,['United States'],['English'],"['Brick Bronsky', 'Lisa Gaye', 'Leesa Rowland'...",['Eric Louzil'],['Bob Mithoff'],"['Lloyd Kaufman', 'Carl Morano', 'Matt Unger']",96.0


In [3]:
## Output of the number of rows and columns respectively.
Moviesdf.shape

(8451, 13)

In [4]:
## The information on the dataset
Moviesdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8451 entries, 0 to 8450
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         8451 non-null   object 
 1   Year         8451 non-null   int64  
 2   kind         8451 non-null   object 
 3   genre        8384 non-null   object 
 4   imdb_rating  8135 non-null   float64
 5   vote         8135 non-null   float64
 6   country      8152 non-null   object 
 7   language     8052 non-null   object 
 8   cast         8280 non-null   object 
 9   director     7140 non-null   object 
 10  composer     5722 non-null   object 
 11  writer       7204 non-null   object 
 12  runtime      6798 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 858.4+ KB


In [5]:
## Missing information within the dataframe
Moviesdf.isnull().sum()

Name              0
Year              0
kind              0
genre            67
imdb_rating     316
vote            316
country         299
language        399
cast            171
director       1311
composer       2729
writer         1247
runtime        1653
dtype: int64

In [6]:
# Drop rows with missing values
#Moviesdf = Moviesdf.dropna()
#Moviesdf.isnull().sum()

In [7]:
Moviesdf.shape

(8451, 13)

## CANDIDATE GENERATION 

In [8]:
## Libaries necessary for the function 
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [9]:
### Helping functions for model 
## Splits a DataFrame into training and test sets
def split_dataframe(df, holdout_fraction=0.2):
    total_size = len(df)
    test_size = int(total_size * holdout_fraction)
    
    # Generate a random boolean array to select rows for the test set
    test_indices = np.random.choice(total_size, size=test_size, replace=False)
    
    # Split the DataFrame into training and test sets based on the selected indices
    test = df.iloc[test_indices]
    train = df.drop(df.index[test_indices])
    
    return train, test

In [10]:
## the libraries necessary for the euclidean distance calculation 
# Select relevant columns for similarity calculation (e.g., movie features)
selected_columns = ['kind', 'genre', 'country','language']  
features = Moviesdf[selected_columns]

In [11]:
# Split the data into training and test sets
training_data, test_data = split_dataframe(Moviesdf, holdout_fraction=0.2)

In [18]:
## The function describes the Euclidian generator 
def euclidean_similarity_generator(training_data, features, k=5):
    
    # Select features for similarity calculation
    training_features = training_data[features]
    
     # Combine categorical features into a single text column
    training_data['combined_features'] = training_features.apply(lambda x: ' '.join(str(feature) for feature in x), axis=1)
    
    # TF-IDF encoding
    vectorizer = TfidfVectorizer()
    training_matrix = vectorizer.fit_transform(training_data['combined_features'])
    
    # Standardize the features
    scaler = StandardScaler(with_mean=False)
    training_features = scaler.fit_transform(training_matrix)
    
    # Calculate pairwise distances between training features
    distances = []
    for i in range(training_features.shape[0]):
        test_movie = training_features[i].toarray().ravel()
        movie_distances = []
        for j in range(training_features.shape[0]):
            if i != j:  # Exclude the same movie from distance calculation
                training_movie = training_features[j].toarray().ravel()
                distance = euclidean(test_movie, training_movie)
                movie_distances.append((distance, j))
        distances.append(movie_distances)
    
    # Sort distances and retrieve indices of most similar movies
    similar_movies_indices = []
    for movie_distances in distances:
        movie_distances.sort()
        indices = [index for _, index in movie_distances[:k]]
        similar_movies_indices.append(indices)
        
      # Transform indices to matching movie names
    #similar_movies = []
    #for indices in similar_movies_indices:
        #movie_names = training_data.iloc[indices]['Name'].tolist()
        #similar_movies.append(movie_names)   
    
     # Transform indices to matching movie names
    similar_movies = []
    for i, indices in enumerate(similar_movies_indices):
        movie_info = {
            'Movie': training_data.iloc[i]['Name'],
            'Similar Movies': training_data.iloc[indices]['Name'].tolist(),
            'Features': training_data.iloc[i][features].tolist()
        }
        similar_movies.append(movie_info)
    
    # Get similar movies' information from training data
    #similar_movies = [training_data.iloc[indices] for indices in similar_movies_indices]
    
    return similar_movies

#### Test call with just the four selected columns.

In [25]:
# Generate similar movies for the training  data.
## I decided to to to focus on a sample size of 1000 first as using the whole training set was taking time 
#and not giving me and output and prompting my notebook to quit. 
from sklearn.utils import resample

sample_size = 1000
# Randomly select a sample of movies from the training dataset
sampled_data = resample(training_data, n_samples=sample_size, random_state=42)

# Generate similar movies for the sampled data
similar_movies = euclidean_similarity_generator(sampled_data, selected_columns)

# Retrieve the first five movies and their similar movies
top_five_movies = similar_movies[:5]

# Create a DataFrame to store the movie information
movie_df = pd.DataFrame(columns=['Movie', 'Similar Movies', 'Features'])

# Populate the DataFrame with the movie information
for movie_info in top_five_movies:
    movie_name = movie_info['Movie']
    similar_movies = movie_info['Similar Movies']
    features = movie_info['Features']
    
    # Create a row for the movie information
    row = {'Movie': movie_name, 'Similar Movies': similar_movies, 'Features': features}
    
    # Append the row to the DataFrame
    movie_df = movie_df.append(row, ignore_index=True)

# Print the DataFrame
movie_df.head()

/tmp/ipykernel_134756/412959957.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/412959957.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/412959957.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/412959957.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/412959957.py:29: FutureWarning: The frame.append method is deprecated and will

,Movie,Similar Movies,Features
0,Little House on the Prairie,"[Little House on the Prairie, Little House on ...","[tv series, ['Drama', 'Family', 'Romance', 'We..."
1,Dance with the Devil,"[Like Water for Chocolate, Japón, Who the Hell...","[movie, ['Action', 'Crime', 'Horror', 'Romance..."
2,Zoolander,"[The Coca-Cola Kid, Madonna: The Girlie Show -...","[movie, ['Comedy'], ['Germany', 'United States..."
3,Joint Security Area,"[The Gingko Bed, No Manners, No Manners, Daria...","[movie, ['Action', 'Drama', 'Thriller', 'War']..."
4,Shikoku,"[Gozu, Female Prisoner Scorpion: Jailhouse 41,...","[movie, ['Horror', 'Thriller'], ['Japan'], ['J..."


#### USING THE FUNCTION TO INCLUDE ALL THE FEATURES BOTH CATEGORICAL AND NUMERICAL 

In [ ]:
# Separate categorical and numerical features
categorical_features = [feature for feature in features if training_data[feature].dtype == 'object']
numerical_features = [feature for feature in features if training_data[feature].dtype != 'object']

In [27]:
## the libraries necessary for the euclidean distance calculation 
# Select relevant columns for similarity calculation (e.g., movie features)
Selected_columns_all = ['Name','Year','kind','genre','imdb_rating','vote','country','language','cast','director', 'composer','writer','runtime']
features = Moviesdf[Selected_columns_all]

In [31]:
# Generate similar movies for the sampled data
similar_movies = euclidean_similarity_generator(sampled_data, Selected_columns_all)

# Retrieve the first five movies and their similar movies
top_five_movies = similar_movies[:5]

# Create a DataFrame to store the movie information
movie_df = pd.DataFrame(columns=['Movie', 'Similar Movies'])

# Populate the DataFrame with the movie information
for movie_info in top_five_movies:
    movie_name = movie_info['Movie']
    similar_movies = movie_info['Similar Movies']
        
    # Create a row for the movie information
    row = {'Movie': movie_name, 'Similar Movies': similar_movies}
    
   # Append the row to the DataFrame
    movie_df = movie_df.append(row, ignore_index=True)

# Print the DataFrame
movie_df.head()

/tmp/ipykernel_134756/601155793.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/601155793.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/601155793.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/601155793.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/601155793.py:19: FutureWarning: The frame.append method is deprecated and will

,Movie,Similar Movies
0,Little House on the Prairie,"[Little House on the Prairie, Little House on ..."
1,Dance with the Devil,"[Jack & the Beanstalk, Tales of the Brothers Q..."
2,Zoolander,"[Jack & the Beanstalk, Tales of the Brothers Q..."
3,Joint Security Area,"[Jack & the Beanstalk, Tales of the Brothers Q..."
4,Shikoku,"[Horses, Jack & the Beanstalk, Tales of the Br..."


In [34]:
# Generate similar movies for the sampled data
similar_movies = euclidean_similarity_generator(sampled_data, Selected_columns_all)

# Retrieve the first five movies and their similar movies
top_five_movies = similar_movies[:5]

# Create a DataFrame to store the movie information
movie_df = pd.DataFrame(columns=['Movie', 'Similar Movies'])

# Populate the DataFrame with the movie information
for movie_info in top_five_movies:
    movie_name = movie_info['Movie']
    similar_movies = movie_info['Similar Movies']
    features = movie_info['Features']
        
     # Create a row for the movie information
    row = {'Movie': movie_name, 'Similar Movies': similar_movies, 'Features': features}
    
   # Append the row to the DataFrame
    movie_df = movie_df.append(row, ignore_index=True)

# Print the DataFrame
movie_df.head()

/tmp/ipykernel_134756/3109462632.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/3109462632.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/3109462632.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/3109462632.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(row, ignore_index=True)
/tmp/ipykernel_134756/3109462632.py:20: FutureWarning: The frame.append method is deprecated and

,Movie,Similar Movies,Features
0,Little House on the Prairie,"[Little House on the Prairie, Little House on ...","[Little House on the Prairie, 1974, tv series,..."
1,Dance with the Devil,"[Jack & the Beanstalk, Tales of the Brothers Q...","[Dance with the Devil, 1997, movie, ['Action',..."
2,Zoolander,"[Jack & the Beanstalk, Tales of the Brothers Q...","[Zoolander, 2001, movie, ['Comedy'], 6.5, 2538..."
3,Joint Security Area,"[Jack & the Beanstalk, Tales of the Brothers Q...","[Joint Security Area, 2000, movie, ['Action', ..."
4,Shikoku,"[Horses, Jack & the Beanstalk, Tales of the Br...","[Shikoku, 1999, movie, ['Horror', 'Thriller'],..."


In [33]:
# Generate similar movies for the sampled data
similar_movies = euclidean_similarity_generator(sampled_data, Selected_columns_all)

# Retrieve the first five movies and their similar movies
top_five_movies = similar_movies[:5]

# Create a DataFrame to store the movie information
movie_df = pd.DataFrame(columns=['Movie', 'Similar Movies', 'Name', 'Year', 'kind', 'genre', 'imdb_rating', 'vote', 'country', 'language', 'cast', 'director', 'composer', 'writer', 'runtime'])

for movie_info in top_five_movies:
    movie_name = movie_info['Movie']
    similar_movies = movie_info['Similar Movies']
    
    # Get the movie information from the training data
    movie_data = training_data.loc[training_data['Name'].isin([movie_name] + similar_movies)]
    
    # Append the movie information to the DataFrame
    movie_df = movie_df.append(movie_data)

# Reset the index of the DataFrame
movie_df.reset_index(drop=True, inplace=True)

# Print the DataFrame
movie_df.head()

/tmp/ipykernel_134756/2047945517.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(movie_data)
/tmp/ipykernel_134756/2047945517.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(movie_data)
/tmp/ipykernel_134756/2047945517.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(movie_data)
/tmp/ipykernel_134756/2047945517.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_df = movie_df.append(movie_data)
/tmp/ipykernel_134756/2047945517.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

,Movie,Similar Movies,Name,Year,kind,genre,imdb_rating,vote,country,language,cast,director,composer,writer,runtime
0,NaN,NaN,Little House on the Prairie,1974,tv series,"['Drama', 'Family', 'Romance', 'Western']",7.5,20775.0,NaN,NaN,"['Melissa Gilbert', 'Michael Landon', 'Karen G...",NaN,NaN,"['Blanche Hanalis', 'Blanche Hanalis', 'Laura ...",90.0
1,NaN,NaN,Little House on the Prairie,1974,tv series,"['Drama', 'Family', 'Romance', 'Western']",7.5,20775.0,NaN,NaN,"['Melissa Gilbert', 'Michael Landon', 'Karen G...",NaN,NaN,"['Blanche Hanalis', 'Blanche Hanalis', 'Laura ...",89.0
2,NaN,NaN,Little House on the Prairie,1974,tv series,"['Drama', 'Family', 'Romance', 'Western']",7.5,20775.0,NaN,NaN,"['Melissa Gilbert', 'Michael Landon', 'Karen G...",NaN,NaN,"['Blanche Hanalis', 'Blanche Hanalis', 'Laura ...",110.0
3,NaN,NaN,Tales of the Brothers Quay,1987,movie,"['Animation', 'Documentary']",8.3,429.0,"['United Kingdom', 'United States']",['English'],NaN,"['Stephen Quay', 'Timothy Quay']",NaN,NaN,86.0
4,NaN,NaN,Little House on the Prairie,1974,tv series,"['Drama', 'Family', 'Romance', 'Western']",7.5,20777.0,NaN,NaN,"['Melissa Gilbert', 'Michael Landon', 'Karen G...",NaN,NaN,"['Blanche Hanalis', 'Blanche Hanalis', 'Laura ...",119.0
